In [14]:
import pandas as pd

In [15]:
df=pd.read_csv("youtube-top-100-songs-2025 (4).csv")

In [16]:
df.head()

,title,fulltitle,description,view_count,categories,tags,duration,duration_string,live_status,thumbnail,channel,channel_url,channel_follower_count
0,ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT. (Official Music Video),ROSÉ & Bruno Mars - APT.\nDownload/stream: ht...,2009014557,Music,YG Entertainment;YG;와이지;K-pop;BLACKPINK;블랙핑크;블...,173,2:53,False,https://i.ytimg.com/vi_webp/ekr2nIex040/maxres...,ROSÉ,https://www.youtube.com/channel/UCBo1hnzxV9rz3...,19200000
1,"Lady Gaga, Bruno Mars - Die With A Smile (Offi...","Lady Gaga, Bruno Mars - Die With A Smile (Offi...",MAYHEM OUT NOW\nhttp://ladygaga.com \n \nListe...,1324833300,Music,Lady Gaga;Bruno Mars;Interscope;Pop,252,4:12,False,https://i.ytimg.com/vi/kPa7bsKwL-c/maxresdefau...,Lady Gaga,https://www.youtube.com/channel/UC07Kxew-cMIay...,29600000
2,Reneé Rapp - Leave Me Alone (Official Music Vi...,Reneé Rapp - Leave Me Alone (Official Music Vi...,"Listen to “BITE ME”, the new album from Reneé ...",2536628,Music,Reneé Rapp;Interscope Records;Pop,160,2:40,False,https://i.ytimg.com/vi/tiPWzFLiz4A/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000
3,Billie Eilish - BIRDS OF A FEATHER (Official M...,Billie Eilish - BIRDS OF A FEATHER (Official M...,Listen to HIT ME HARD AND SOFT: https://billie...,558329099,Music,Billie Eilish;Darkroom/Interscope Records;Alte...,231,3:51,False,https://i.ytimg.com/vi/V9PVRfjEBTI/maxresdefau...,Billie Eilish,https://www.youtube.com/channel/UCDGmojLIoWpXo...,56800000
4,Reneé Rapp - Mad (Official Music Video),Reneé Rapp - Mad (Official Music Video),"Listen to “BITE ME”, the new album from Reneé ...",2113548,Music,Reneé Rapp;Interscope Records;Pop,180,3:00,False,https://i.ytimg.com/vi/xkWQM3flsiY/maxresdefau...,Reneé Rapp,https://www.youtube.com/channel/UCZy4ki_L4bzw9...,408000


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   100 non-null    object
 1   fulltitle               100 non-null    object
 2   description             100 non-null    object
 3   view_count              100 non-null    int64 
 4   categories              100 non-null    object
 5   tags                    85 non-null     object
 6   duration                100 non-null    int64 
 7   duration_string         100 non-null    object
 8   live_status             100 non-null    bool  
 9   thumbnail               100 non-null    object
 10  channel                 100 non-null    object
 11  channel_url             100 non-null    object
 12  channel_follower_count  100 non-null    int64 
dtypes: bool(1), int64(3), object(9)
memory usage: 9.6+ KB


In [18]:
df["categories"].unique()

array(['Music', 'People & Blogs'], dtype=object)

# Data Preprocassing

In [19]:
import pandas as pd

class Cleaner:
    def __init__(self, df):
        self.df = df

    def tozala(self):
        for col in self.df.columns:
            if self.df[col].isnull().any():
                if self.df[col].dtype == 'object':
                    self.df[col] = self.df[col].fillna(self.df[col].mode()[0])
                else:
                    self.df[col] = self.df[col].fillna(self.df[col].mean())
        return self

    def get_df(self):
        return self.df

In [22]:
cleaner = Cleaner(df)       
cleaned_df = cleaner.tozala().get_df()

In [54]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np

class Encoder:
    def __init__(self, df):
        self.df = df.copy()
        self.label_encoders = {}

    def encodla(self):
        for col in self.df.columns:
            # faqat object turlarini ishlov beramiz
            if self.df[col].dtype == 'object':
                self.df[col] = self.df[col].fillna('missing')

                # juda uzun matnli ustunlar uchun (masalan description) encoding foydasiz bo‘ladi
                # Shuning uchun faqat qisqa kategoriyalarni encode qilamiz:
                if self.df[col].nunique() <= 20:
                    if self.df[col].nunique() <= 5:
                        dummies = pd.get_dummies(self.df[col], prefix=col, dtype=int)
                        self.df = pd.concat([self.df.drop(columns=[col]), dummies], axis=1)
                    else:
                        le = LabelEncoder()
                        self.df[col] = le.fit_transform(self.df[col].astype(str))
                        self.label_encoders[col] = le
                else:
                    # juda ko‘p unikal qiymatli ustunlar uchun
                    # (masalan title, description, tags, url) encoding qilmaslik maqsadga muvofiq
                    print(f"⚠️  Ustun '{col}' juda ko‘p noyob qiymatlarga ega ({self.df[col].nunique()}) — encoding o'tkazilmadi.")
        return self.df

    def get_df(self):
        return self.df


# 🔹 Foydalanish:
encoder = Encoder(df)
df_encoded = encoder.encodla()

print(df_encoded.head())


⚠️  Ustun 'title' juda ko‘p noyob qiymatlarga ega (100) — encoding o'tkazilmadi.
⚠️  Ustun 'fulltitle' juda ko‘p noyob qiymatlarga ega (100) — encoding o'tkazilmadi.
⚠️  Ustun 'description' juda ko‘p noyob qiymatlarga ega (100) — encoding o'tkazilmadi.
⚠️  Ustun 'tags' juda ko‘p noyob qiymatlarga ega (76) — encoding o'tkazilmadi.
⚠️  Ustun 'duration_string' juda ko‘p noyob qiymatlarga ega (69) — encoding o'tkazilmadi.
⚠️  Ustun 'thumbnail' juda ko‘p noyob qiymatlarga ega (100) — encoding o'tkazilmadi.
⚠️  Ustun 'channel' juda ko‘p noyob qiymatlarga ega (65) — encoding o'tkazilmadi.
⚠️  Ustun 'channel_url' juda ko‘p noyob qiymatlarga ega (69) — encoding o'tkazilmadi.
                                               title  \
0   ROSÉ & Bruno Mars - APT. (Official Music Video)   
1  Lady Gaga, Bruno Mars - Die With A Smile (Offi...   
2  Reneé Rapp - Leave Me Alone (Official Music Vi...   
3  Billie Eilish - BIRDS OF A FEATHER (Official M...   
4            Reneé Rapp - Mad (Official Music

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   title                   100 non-null    object
 1   fulltitle               100 non-null    object
 2   description             100 non-null    object
 3   view_count              100 non-null    int64 
 4   categories              100 non-null    object
 5   tags                    100 non-null    object
 6   duration                100 non-null    int64 
 7   duration_string         100 non-null    object
 8   live_status             100 non-null    bool  
 9   thumbnail               100 non-null    object
 10  channel                 100 non-null    object
 11  channel_url             100 non-null    object
 12  channel_follower_count  100 non-null    int64 
dtypes: bool(1), int64(3), object(9)
memory usage: 9.6+ KB


In [7]:
from sklearn.linear_model import LogisticRegression
model_lreg = LogisticRegression()
from sklearn.tree import DecisionTreeClassifier
model_dtree = DecisionTreeClassifier()
from sklearn.ensemble import RandomForestClassifier
model_ranfr = RandomForestClassifier()
from sklearn.neighbors import KNeighborsClassifier
model_knreg = KNeighborsClassifier()
from sklearn.svm import SVC
model_svc = SVC()

In [9]:
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix,classification_report,log_loss,matthews_corrcoef,cohen_kappa_score

# 🧮 Classification Evaluation Metrics in Machine Learning

Quyidagi jadvalda mashinaviy o‘rganishdagi **classification (tasniflash)** modellarini baholash uchun ishlatiladigan asosiy **metrikalar**, ularning **ma’nosi** va **`scikit-learn` kutubxonasidagi chaqirish kodi** keltirilgan.

| № | Metric nomi              | Ma’nosi (tavsif)                                                                 | `sklearn` da chaqirish kodi                            |
|---|---------------------------|----------------------------------------------------------------------------------|---------------------------------------------------------|
| 1 | **Accuracy**              | To‘g‘ri bashoratlar ulushi (umumiy natijalarning necha foizi to‘g‘ri)            | `from sklearn.metrics import accuracy_score`            |
| 2 | **Precision**             | Model “Positive” deb aytganda, qanchasi aslida ham “Positive” bo‘lganini o‘lchaydi | `from sklearn.metrics import precision_score`           |
| 3 | **Recall (Sensitivity)**  | Haqiqiy “Positive” namunalar orasida model nechta to‘g‘ri topganini o‘lchaydi     | `from sklearn.metrics import recall_score`              |
| 4 | **F1-Score**              | Precision va Recall o‘rtasidagi muvozanat (ularning harmonic mean)                | `from sklearn.metrics import f1_score`                  |
| 5 | **ROC-AUC Score**         | Modelning Positive/Negative ajratish qobiliyatini ko‘rsatadi                      | `from sklearn.metrics import roc_auc_score`             |
| 6 | **Confusion Matrix**      | To‘g‘ri va noto‘g‘ri tasniflangan namunalar sonini ko‘rsatadi                     | `from sklearn.metrics import confusion_matrix`          |
| 7 | **Classification Report** | Precision, Recall, F1-score va Accuracy’ni jamlab chiqaradi                       | `from sklearn.metrics import classification_report`     |
| 8 | **Log Loss (Cross-Entropy)** | Model ishonchliligi (probability) asosida xatolikni o‘lchaydi                  | `from sklearn.metrics import log_loss`                  |
| 9 | **Matthews Corr. Coeff.** | Barcha toifalarning muvozanatli natijasini o‘lchaydi (balansli metrika)           | `from sklearn.metrics import matthews_corrcoef`         |
|10 | **Cohen’s Kappa**         | Model natijalarini tasodifiy taxmin bilan solishtirib baholaydi                    | `from sklearn.metrics import cohen_kappa_score`         |


In [11]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

# 📊 Regression Evaluation Metrics in Machine Learning

Quyidagi jadvalda **regression (bashoratlash)** modellarining sifatini baholash uchun ishlatiladigan asosiy **metrikalar**, ularning **ma’nosi** va **`scikit-learn`dagi chaqirish kodi** keltirilgan.

| № | Metric nomi             | Ma’nosi (tavsif)                                                                 | `sklearn` da chaqirish kodi                           |
|---|--------------------------|----------------------------------------------------------------------------------|--------------------------------------------------------|
| 1 | **Mean Absolute Error (MAE)** | Bashorat xatoliklarining o‘rtacha miqdorini (modul bo‘yicha) o‘lchaydi. Kichik bo‘lishi yaxshi. | `from sklearn.metrics import mean_absolute_error`      |
| 2 | **Mean Squared Error (MSE)**  | Xatoliklarning kvadratlari o‘rtachasi. Katta xatolarni ko‘proq jazolaydi.      | `from sklearn.metrics import mean_squared_error`       |
| 3 | **Root Mean Squared Error (RMSE)** | MSE ning ildizi — xatolikni asl birlikda ko‘rsatadi.                         | `np.sqrt(mean_squared_error(y_test, y_pred))`          |
| 4 | **R² Score (Coefficient of Determination)** | Model ma’lumotlardagi dispersiyani qanchalik tushuntira olishini ko‘rsatadi.  | `from sklearn.metrics import r2_score`                 |
| 5 | **Adjusted R² Score**          | R² ni tuzatilgan versiyasi, xususiyatlar sonini hisobga oladi.                 | `1 - (1 - r2) * ((n - 1) / (n - p - 1))` (formuladan) |
| 6 | **Mean Absolute Percentage Error (MAPE)** | Xatolikni foiz shaklida o‘lchaydi (foizdagi o‘rtacha farq).                  | `from sklearn.metrics import mean_absolute_percentage_error` |
| 7 | **Median Absolute Error**      | Median asosida xatolikni o‘lchaydi, chiqindilarga kam sezgir.                 | `from sklearn.metrics import median_absolute_error`    |
| 8 | **Explained Variance Score**   | Model ma’lumotlardagi o‘zgarishni qanchalik tushuntira olishini o‘lchaydi.     | `from sklearn.metrics import explained_variance_score` |
| 9 | **Max Error**                  | Eng katta individual xatolikni o‘lchaydi.                                     | `from sklearn.metrics import max_error`                |
|10 | **Mean Bias Error (MBE)**      | Modelning o‘rtacha “yuqori” yoki “past” bashorat qilganini bildiradi.         | `np.mean(y_pred - y_test)`                            |
